In [ ]:
import pandas as pd
import numpy as np

data =  open('links.txt').read().splitlines()
PATH = 'data/'
data



In [ ]:
dfs = [pd.read_csv(PATH + path) for path in data]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.columns = df.columns.str.strip()


In [ ]:
df['DATETIME'] = pd.to_datetime(df.DATE + " " + df.TIME, format='%m/%d/%Y %H:%M:%S')


In [ ]:
# Make sure there are no duplicate entries
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)

In [ ]:
df[["PREV_DATETIME", "PREV_ENTRIES", "PREV_EXITS"]] = (df.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATETIME", "ENTRIES", "EXITS"]
                                            .transform(lambda grp: grp.shift(1)))
df.head(5)

In [ ]:
df.dropna(subset=["PREV_DATETIME"], axis=0, inplace=True)

In [ ]:
df['TIME_INTERVAL'] = df['DATETIME'] - df['PREV_DATETIME']
df = df.drop(df[(df['TIME_INTERVAL'] > '05:00:00') & (df['TIME_INTERVAL'] < '03:00:00')].index)

In [ ]:
# modify entry counts that are negative and remove the ones with outrageous values
df['ENTRY_COUNT'] = df.ENTRIES - df.PREV_ENTRIES
df.ix[df.ENTRY_COUNT < 0, 'ENTRY_COUNT'] =  - df['ENTRY_COUNT']
df = df.drop(df[df.ENTRY_COUNT > 1000000].index)

In [ ]:
# modify exit counts that are negative and remove the ones with outrageous values
df['EXIT_COUNT'] = df.EXITS - df.PREV_EXITS
df.ix[df.EXIT_COUNT < 0, 'EXIT_COUNT'] =  - df['EXIT_COUNT']
df = df.drop(df[df.EXIT_COUNT > 1000000].index)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
df['WEEKDAY'] = df['DATE'].dt.dayofweek
df['TIME'] = pd.to_datetime(df['TIME'], format='%H:%M:%S')
df['HOUR'] = df['TIME'].dt.hour

In [ ]:
df = df[(df['DATE'] >= '05/01/2016') & (df['DATE'] < '07/01/2016')]

In [ ]:
import csv
# Merges same station
df['COMPLETE_NAME'] = df['STATION'] + ' ' + df['LINENAME']
with open('name_mapping.csv') as namefile:
    reader = csv.reader(namefile)
    for row in reader:
        df.loc[df.COMPLETE_NAME == row[0], 'COMPLETE_NAME'] = row[1]

In [ ]:
# Final info on median income and percentage of tech people
df_income = pd.read_csv('final_merge_info.csv')

In [ ]:
final_df = pd.merge(df, df_income, on=['COMPLETE_NAME'])

In [ ]:
# divides data to 2 dfs
df_weekday = final_df[(final_df.WEEKDAY < 3 )|(final_df.WEEKDAY > 4)]
df_weekend = final_df[(final_df.WEEKDAY > 2 )&(final_df.WEEKDAY < 5 )]

In [ ]:
# divides by 100 because value is in percentage
df_weekend.loc['Information_Industry_%'] = df_weekend['Information_Industry_%'] / 100

In [ ]:
# Get entry data based on percentage of tech people
df_weekend['ENTRY_TECH_%'] = df_weekend['ENTRY_COUNT'] * df_weekend['Information_Industry_%']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline
import seaborn as sns


In [ ]:
# Sum up entry counts daily, and take the average
df_weekend_daily_entries = df_weekend.groupby(['COMPLETE_NAME', 'DATE'], as_index=False)['ENTRY_COUNT','ENTRY_TECH_%'].sum()
df_weekend_daily_entries = df_weekend_daily_entries.groupby(['COMPLETE_NAME'], as_index=False)['ENTRY_COUNT', 'ENTRY_TECH_%'].mean()

In [ ]:
df_weekend_top = df_weekend_daily_entries.sort_values(by=['ENTRY_TECH_%'], ascending=False).head(20)

sns.barplot(x='COMPLETE_NAME', y='ENTRY_TECH_%', data=df_weekend_top)
ax = plt.axes()
ax.set_title('Top stations on the weekends with people in the tech industry')
ax.set_ylabel('Daily entries')
for l in ax.get_xticklabels():
    l.set_rotation(90)

In [ ]:
df_weekday_morning = df_weekday[(df_weekday.HOUR > 6) & (df_weekday.HOUR < 13)]
df_weekday_morning = df_weekday_morning.groupby(['COMPLETE_NAME', 'DATE'], as_index=False)['ENTRY_COUNT', 'EXIT_COUNT'].sum()
df_weekday_morning = df_weekday_morning.groupby(['COMPLETE_NAME'], as_index=False).mean()
df_weekday_morning['EXIT_ENTRY_RATIO'] = df_weekday_morning['EXIT_COUNT'] / df_weekday_morning['ENTRY_COUNT']

mask = ((df_weekday_morning['EXIT_COUNT'] < 20000) & (df_weekday_morning['EXIT_ENTRY_RATIO'] < 10)
        & (df_weekday_morning['EXIT_COUNT'] > 5000) & (df_weekday_morning['EXIT_ENTRY_RATIO']> 2))

In [ ]:
df_weekend_fave = pd.merge(df_weekday_morning,df_weekend, on=['COMPLETE_NAME'])
df_weekend_fave.columns

In [ ]:
mask = ((df_weekend_fave['ENTRY_TECH_%'] <40000) & (df_weekend_fave['ENTRY_EXIT_RATIO'] < 20)
        & (df_weekend_fave['ENTRY_TECH_%'] >10000) & (df_weekend_fave['ENTRY_EXIT_RATIO'] > 2))
g = sns.lmplot('ENTRY_EXIT_RATIO', 'ENTRY_TECH_%', 
           data=df_weekend_fave[mask], 
           fit_reg=False,   
           scatter_kws={"marker": "D", 
                        "s": 10},
           hue='COMPLETE_NAME',
          )
g.set_axis_labels("Entry:Exit", "Entry Count")

In [ ]:
length = len(df_weekend_fave)
df_weekend_fave['ratio_quantile'] = df_weekend_fave['ENTRY_EXIT_RATIO'].rank() / length
df_weekend_fave['tech_quantile'] = df_weekend_fave['ENTRY_TECH_%'].rank() / length
df_weekend_fave.head(4)

In [ ]:
df_weekend_fave['total_score'] = df_weekend_fave['ratio_quantile'] + df_weekend_fave['tech_quantile']
df_weekend_fave.sort_values(by=['tech_quantile'], ascending=False).head(5)['COMPLETE_NAME']

In [ ]:
df_residential['COMPLETE_NAME']

In [ ]:
df_residential = pd.merge(df_residential, df_weekend, how='left')
df_residential = df_weekend.mask(df_weekend['COMPLETE_NAME'].isin(df_residential.STATION))

In [ ]:
df_residential_top_times = df_residential.groupby(['WEEKDAY', 'HOUR'], as_index=False)['ENTRY_COUNT'].mean()

df_saturday = df_residential_top_times[df_residential_top_times.WEEKDAY == 3]
df_sunday = df_residential_top_times[df_residential_top_times.WEEKDAY == 4]
sns.barplot(x='HOUR', y='ENTRY_COUNT', data=df_saturday)
ax = plt.axes()
ax.set_title('Entry count on Saturday in Residential Areas')
ax.set_ylabel('Entry Count')
for l in ax.get_xticklabels():
    l.set_rotation(90)